# Modelado de Tópicos

En este notebook presentaremos una técnica de aprendizaje no supervisado dentro del campo de procesamiento de lenguaje natural llamado Modelado de Tópicos (Topic Modeling). Esta técnica se busca en construir/identificar temas en base a las distribuciones de las palabras en un conjunto de documentos.

## Conjunto de datos

El conjunto de datos que trataremos en el presente notebook será un conjunto de plots (tramas) de aproximadamente 35,000 películas de Wikipedia.

Referencia: https://www.kaggle.com/jrobischon/wikipedia-movie-plots

In [1]:
# Importamos pandas
import pandas as pd

In [2]:
# Cargamos el dataset a pandas
df = pd.read_csv("wiki_movie_plots_deduped.csv")
# Dimensiones del dataframe
df.shape

(34886, 8)

In [3]:
df.sample(5)

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
21057,2009,Triangle,British,Christopher Smith,"Melissa George, Liam Hemsworth, Rachael Carpani",horror,https://en.wikipedia.org/wiki/Triangle_(2009_B...,While preparing to take her autistic son Tommy...
26406,2006,Taxi No. 9211,Bollywood,Unknown,"John Abraham, Nana Patekar, Sameera Reddy, Son...","social, thriller",https://en.wikipedia.org/wiki/Taxi_No._9211,Taxi No. 9 2 11 focuses on Raghav Shastri (Nan...
23937,2003,Bhalo Theko,Bengali,Goutam Halder,"Soumitra Chatterjee, Vidya Balan",unknown,https://en.wikipedia.org/wiki/Bhalo_Theko,The film is set in Acharya Jagadish Chandra Bo...
19343,1957,Sea Wife,British,Bob McNaught,"Joan Collins, Richard Burton",thriller,https://en.wikipedia.org/wiki/Sea_Wife,Michael Cannon (Richard Burton) returns to Lon...
29572,1978,Thappu Thalangal,Tamil,K. Balachander,"Rajinikanth, Saritha",unknown,https://en.wikipedia.org/wiki/Thappu_Thalangal,"Devu, a local thug whose weapon of choice is a..."


In [ ]:
# Información de los géneros
df["Genre"].value_counts()

## Limpieza de datos

Haremos una pequeña limpieza de datos. Para esta ocasión nos ayudaremos de la librería spaCy. Filtraremos los "plots" que estén compuesto por palabras que contengan exclusivamente letras, que no sean "stopwords" y que no sean nombres propios. Luego obtendremos el lema de la palabra.

In [ ]:
# Importamos la librería spaCy
import spacy

In [ ]:
# Descargamos los paquetes
!python -m spacy download en_core_web_md

In [ ]:
# Cargamos los modelos
nlp = spacy.load("en_core_web_md")

In [ ]:
# Ahora definiremos nuestra función que nos ayudará a limpiar nuestros datos
def preprocess_text(text):
    tokens = []
    doc = nlp(text)

    for token in doc:
        if ((token.is_alpha) and (not token.is_stop) and (token.pos_ != "PROPN")):
            tokens.append(token.lemma_)

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [ ]:
# Utilizaremos esta librería que nos ayudará a "estimar" los tiempos de ejecución de algunas
# instrucciones
from tqdm import tqdm
tqdm.pandas()

In [ ]:
# Aplicamos la función de preprocesamiento a cada "Plot" del dataframe
# Esta tarea puede demorar varios minutos (45 min)
df["plot_preprocessed"] = df["Plot"].progress_apply(preprocess_text)

In [ ]:
df.sample(5)[["Plot", "plot_preprocessed"]]

## Exploración de los datos

In [ ]:
# Importamos algunas librerías que nos ayudará a explorar los datos
import nltk
from nltk import FreqDist
from nltk import word_tokenize

nltk.download("punkt")

In [ ]:
# Obtenemos la distribución de los tokens
tokens_distribution = FreqDist(word_tokenize(' '.join(df["plot_preprocessed"])))

In [ ]:
# Los tokens más comunes
tokens_distribution.most_common(10)

In [ ]:
# Número de tokens
tokens_distribution.N()

In [ ]:
# Tamaño del vocabularios
len(tokens_distribution)

## Latent Dirichlet Allocation (LDA)

Es un modelo estadístico generativo que se utiliza como una técnica de modelado de temas que puede clasificar el texto de un documento en un tema en particular.

Este modelo tiene una implementación en Scikit-Learn.

Enlace: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html

In [ ]:
# Importamos CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Vectorizaremos los documentos, haremos un "Bag of Words" (BoW)
# min_df: ignora a los términos que aparecen en menos de un número de documentos
bow = CountVectorizer(min_df=10).fit(df["plot_preprocessed"])
plot_bow = bow.transform(df["plot_preprocessed"])

In [ ]:
# Dimensiones de nuestros BoW
plot_bow.shape

In [ ]:
# Importamos la implementación LDA de Scikit-Learn
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
# Instanciamos nuestro LDA
# Parámetros:
#   n_components: número de tópicos/temas
#   max_iter: el número máximo de "pasos" sobre la data de entrenamiento (épocas)
#   learning_method:
#       batch: Método de Bayes variacional por batches. Usar todos los datos de entrenamiento en cada actualización.
#       online: Método de Bayes variacional en línea. Usa mini-batches de datos de entrenamiento en cada actualización.
#   batch_size: número de documentos a utilizar en cada actualización/iteración
lda = LatentDirichletAllocation(n_components=10, max_iter=15, learning_method="online", batch_size=256, verbose=True)

In [ ]:
%%time
# Alimentamos nuestro modelo LDA con la data de "plots"
lda.fit(plot_bow)

In [ ]:
# Componentes
lda.components_

In [ ]:
# Número de componentes, número de tópicos
len(lda.components_)

In [ ]:
# Número de "variables" por cada componente
len(lda.components_[0])

¿Cómo saber las palabras dentro de cada tópico?

In [ ]:
for id, topic in enumerate(lda.components_):
    # Ordenamos los tópicos de menor a mayor y obtenemos las 10 últimas palabras
    words = [bow.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]]
    print("Topic {}: {}".format(id, ' '.join(words)))

¿Cómo saber los tópicos por un documento en específico?

In [ ]:
# Importamos algunas librerías que nos ayudará a la visualización de los datos
import seaborn as sns
import numpy as np

In [ ]:
# Película a buscar
movie = df.loc[df["Title"] == "Coco"]
movie

In [ ]:
# Pesos por cada tópico (por ciento)
movie_topics = lda.transform(plot_bow[movie.index]).flatten() * 100
movie_topics

In [ ]:
# Visualizamos los tópicos para la película "Coco"
sns.barplot(x=np.arange(len(movie_topics)), y=movie_topics)

¿Cómo saber a que tópico pertenece cada documento del dataset?

In [ ]:
# Para cada plot, obtenemos el tópico que tiene mayor porcentaje
topics = [lda.transform(plot).argsort()[0][-1] for plot in plot_bow]

In [ ]:
# Visualizamos los documentos por tópicos
sns.countplot(x=topics)

## Evaluación del modelo

Podemos usar la puntuación de coherencia en el modelado de tópicos para medir qué tan interpretables son los temas para los humanos. En este caso, los temas se representan como las primeras N palabras con mayor probabilidad de pertenecer a ese tema en particular. En otras palabras, el puntaje de coherencia mide qué tan similares son estas palabras entre sí.

In [ ]:
# Instalamos la librería que nos ayudará a computar la métrica de coherencia
!pip install tmtoolkit

In [ ]:
# Importamos la métrica
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

In [ ]:
# Calculamos la métrica para los tópicos
# Parámatros:
#     measure: 'c_v' coherence value
#     top_n: número de las palabras más probables por tópico
#     topic_word_distrib: distribución de palabras por tópico, dimensiones KxM, donde K es el número de tópicos y
#                           M es el tamaño de vocabulario
#     vocab: array o lista del vocabulario
#     texts: lista de los documentos tokenizados
metric_coherence_gensim(measure="c_v", top_n=10, topic_word_distrib=lda.components_,
                        vocab=np.array([key for key in bow.vocabulary_.keys()]),
                        texts=[word_tokenize(plot) for plot in df["plot_preprocessed"]])

In [ ]:
for id, topic in enumerate(lda.components_):
    # Ordenamos los tópicos de menor a mayor y obtenemos las 10 últimas palabras
    words = [bow.get_feature_names_out()[i] for i in topic.argsort()[:-11:-1]]
    print("Topic {}: {}".format(id, ' '.join(words)))